In [ ]:
import sys
sys.path.append('/home/ubuntu/Nilay/PersonReIDModels/LUPerson-extended/fast-reid')
from demo_visualizer import Visualizer
from predictor import FeatureExtractionDemo
from fastreid.config import get_cfg
from datasets import load_accumulated_info_of_dataset

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/home/ubuntu/Nilay/PersonReIDModels/LUPerson-extended/fast-reid/configs/LaST/mgn_R50_moco_cache_test.yml")
cfg.freeze()

demo = FeatureExtractionDemo(cfg)

In [ ]:
import os
import numpy as np
def load_saved_feat(dir=None):
    assert os.path.isdir(dir), f"{dir} doesn't exist"
    feat_file = os.path.join(dir, "feat.npy")
    pid_file = os.path.join(dir, "pid.npy")
    camid_file = os.path.join(dir, "camid.npy")
    imgpath_file = os.path.join(dir, "imgpath.npy")

    assert os.path.isfile(feat_file), "Features not found"
    assert os.path.isfile(pid_file), "Pids not found"
    assert os.path.isfile(camid_file), "Camids not found"

    if os.path.isfile(imgpath_file):
        print("File containing saved image paths found")
        return (np.load(feat_file, allow_pickle=True).astype(np.float32),\
             np.load(pid_file, allow_pickle=True).astype(np.float32), np.load(camid_file, allow_pickle=True), np.load(imgpath_file, allow_pickle=True).tolist())    

    return (np.load(feat_file, allow_pickle=True).astype(np.float32),\
         np.load(pid_file, allow_pickle=True).astype(np.float32), np.load(camid_file, allow_pickle=True), None)

In [ ]:
import pandas as pd
root_folder = "/home/ubuntu/Nilay/PersonReIDModels/Datasets"
dataset_name = "LaST"

_, query, gallery = load_accumulated_info_of_dataset(root_folder, dataset_name, use_eval_set=False, verbose=True)
num_query = pd.DataFrame(query)["image_file_path"].nunique()
test_dataset =  query + gallery
print("Done")

In [ ]:
from demo_visualizer import Visualizer
input_dir = '/home/ubuntu/Nilay/PersonReIDModels/qualitative_evaluation/cache/LUPerson-extended/trained/'
dataset_name = 'LaST'

feats, pids, camids, imgpaths = load_saved_feat(os.path.join(input_dir, dataset_name))
q_feats = feats[:num_query]
g_feats = feats[num_query:]
q_pids = np.asarray(pids[:num_query])
g_pids = np.asarray(pids[num_query:])
q_camids = np.asarray(camids[:num_query])
g_camids = np.asarray(camids[num_query:])

visualizer = Visualizer(test_dataset, None, None, num_query=num_query)
visualizer.index_gallery(g_feats=g_feats, g_pids=g_pids, g_camids=g_camids)

In [ ]:
from fastreid.utils.file_io import PathManager
from PIL import Image
import torchvision.transforms as T
from fastreid.data.transforms import *

def read_image(file_name, format=None):
    """
    Read an image into the given format.
    Will apply rotation and flipping if the image has such exif information.
    Args:
        file_name (str): image file path
        format (str): one of the supported image modes in PIL, or "BGR"
    Returns:
        image (np.ndarray): an HWC image
    """
    with PathManager.open(file_name, "rb") as f:
        image = Image.open(f)

        # capture and ignore this bug: https://github.com/python-pillow/Pillow/issues/3973
        try:
            image = ImageOps.exif_transpose(image)
        except Exception:
            pass

        if format is not None:
            # PIL only supports RGB, so convert to RGB and flip channels over below
            conversion_format = format
            if format == "BGR":
                conversion_format = "RGB"
            image = image.convert(conversion_format)
        image = np.asarray(image)
        if format == "BGR":
            # flip channels if needed
            image = image[:, :, ::-1]
        # PIL squeezes out the channel dimension for "L", so make it HWC
        if format == "L":
            image = np.expand_dims(image, -1)
        image = Image.fromarray(image)
        return image

def read_image_url(image, format=None):
    """
    Read an image into the given format.
    Will apply rotation and flipping if the image has such exif information.
    Args:
        file_name (str): image file path
        format (str): one of the supported image modes in PIL, or "BGR"
    Returns:
        image (np.ndarray): an HWC image
    """
    # capture and ignore this bug: https://github.com/python-pillow/Pillow/issues/3973
    try:
        image = ImageOps.exif_transpose(image)
    except Exception:
        pass

    if format is not None:
        # PIL only supports RGB, so convert to RGB and flip channels over below
        conversion_format = format
        if format == "BGR":
            conversion_format = "RGB"
        image = image.convert(conversion_format)
    image = np.asarray(image)
    if format == "BGR":
        # flip channels if needed
        image = image[:, :, ::-1]
    # PIL squeezes out the channel dimension for "L", so make it HWC
    if format == "L":
        image = np.expand_dims(image, -1)
    image = Image.fromarray(image)
    return image

def build_transforms(cfg):
    res = []
    size_test = cfg.INPUT.SIZE_TEST
    res.append(T.Resize(size_test, interpolation=3))
    res.append(ToTensor())
    return T.Compose(res)

In [ ]:
## Importing Necessary Modules
import requests # to get image from the web
import shutil # to save it locally

## Set up the image URL and filename
# image_url = "https://cdn.pixabay.com/photo/2020/02/06/09/39/summer-4823612_960_720.jpg"
image_url = "https://fashionsizzle.com/wp-content/uploads/2015/12/will-smith-premiere-after-earth-02.jpg"
filename = image_url.split("/")[-1]

# Open the url image, set stream to True, this will return the stream content.
r = requests.get(image_url, stream = True)

# Check if the image was retrieved successfully
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
    with open(filename,'wb') as f:
        shutil.copyfileobj(r.raw, f)
        
    print('Image sucessfully Downloaded: ',filename)
else:
    print('Image Couldn\'t be retreived')

In [ ]:
import cv2
import requests
from io import BytesIO
url = "https://i.pinimg.com/736x/af/bb/fc/afbbfc9c5cf27a87b8a1b7d05cfd9813--rihanna-red-dress-rihanna-grammys.jpg"
img = Image.open(requests.get(url, stream=True).raw)

query_name = "Sample Input"

image_content = read_image_url(img)
transform = build_transforms(cfg)
image_content = transform(image_content)
image_content = image_content[None, :]
q_feat = demo.predictor(image_content).numpy()

visualizer.get_top_10(query_name, img, q_feat)

In [ ]:
import cv2
query_name = "Sample Input"
query_img_file_path = f'/home/ubuntu/Nilay/PersonReIDModels/LUPerson-extended/fast-reid/demo/{filename}'
print(len(test_dataset))
image_content = read_image(query_img_file_path)
transform = build_transforms(cfg)
image_content = transform(image_content)
image_content = image_content[None, :]
print(image_content.shape)
q_feat2 = demo.predictor(image_content).numpy()
q_feat = np.load("/home/ubuntu/Nilay/PersonReIDModels/qualitative_evaluation/cache/LUPerson-extended/trained/LaST/feat.npy")[0]
q_feat = q_feat.reshape(1, -1)
print(q_feat2)
print(q_feat)
visualizer.get_top_10(query_name, cv2.imread(query_img_file_path), q_feat2)